# Heston Surrogate Pricer - Complete Pipeline

## 🎯 Project Overview

This notebook demonstrates the complete training and evaluation pipeline for the **Heston surrogate pricing model**. Traditional option pricing under the Heston stochastic volatility model requires computationally expensive Fast Fourier Transform (FFT) methods. Our approach replaces this with a fast, accurate neural network that learns to predict implied volatility surfaces directly from Heston parameters.

### 📚 Theoretical Background

The **Heston Model** (1993) describes asset price dynamics with stochastic volatility:

$$
\begin{align}
dS_t &= rS_t dt + \sqrt{v_t}S_t dW_1^t \\
dv_t &= \kappa(\theta - v_t)dt + \sigma\sqrt{v_t}dW_2^t
\end{align}
$$

where:
- $S_t$: Asset price at time $t$
- $v_t$: Instantaneous variance at time $t$
- $r$: Risk-free rate
- $\kappa$: Mean reversion speed
- $\theta$: Long-term variance level
- $\sigma$: Volatility of volatility (vol-of-vol)
- $\rho = dW_1^t \cdot dW_2^t$: Correlation between price and volatility shocks

### 🚀 Surrogate Model Approach

Instead of solving the complex pricing integral:
$$C(K,T) = e^{-rT} \mathbb{E}[\max(S_T - K, 0)]$$

We train a neural network to learn the direct mapping:
$$f_{\text{NN}}: (v_0, \kappa, \theta, \sigma, \rho, r, K, T) \mapsto \text{IV}(K,T)$$

This provides **~1000x speedup** over traditional FFT methods while maintaining high accuracy.

## 📋 Pipeline Overview

1. **🔧 Environment Setup**: Import libraries and configure reproducibility
2. **⚙️ Configuration**: Define hyperparameters and training settings
3. **📊 Data Loading**: Load preprocessed Heston parameter-IV surface pairs
4. **🎯 PCA Analysis**: Reduce output dimensionality while preserving structure
5. **🏗️ Model Architecture**: Build ResidualMLP with advanced loss function
6. **🚂 Model Training**: Train with callbacks and monitoring
7. **📈 Training Analysis**: Visualize learning curves and convergence
8. **🧪 Model Evaluation**: Comprehensive test set evaluation
9. **🎯 Bucket Analysis**: Performance across different market regions
10. **📊 Visualization**: IV surface comparisons and error analysis
11. **🔍 Statistical Analysis**: Residual distribution and normality tests
12. **💾 Results Export**: Save artifacts and generate comprehensive reports

## 1. 🔧 Setup and Imports

### 📚 Library Dependencies

We begin by importing all necessary libraries for our machine learning pipeline:

- **Core Scientific Computing**: `numpy`, `scipy` for mathematical operations
- **Data Analysis**: `pandas` for data manipulation
- **Machine Learning**: `tensorflow`, `keras` for neural network implementation
- **Dimensionality Reduction**: `sklearn.decomposition.PCA` for output compression
- **Visualization**: `matplotlib`, `seaborn` for plotting and analysis
- **Utilities**: `pickle`, `json` for serialization and configuration

### 🎯 Project Module Imports

Our custom modules provide specialized functionality:

- **`model_architectures`**: ResidualMLP implementation, PCA utilities, advanced loss functions
- **`training_utils`**: Reproducibility setup, callback creation, artifact management

### 🔒 Reproducibility Setup

Ensuring reproducible results is crucial for scientific validity. We'll set random seeds for:
- **NumPy**: `np.random.seed()`
- **TensorFlow**: `tf.random.set_seed()`
- **Python**: `random.seed()`

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from pathlib import Path
from datetime import datetime

# Machine Learning
import tensorflow as tf
import keras
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import project modules
from src.model_architectures import (
    build_resmlp_pca_model,
    fit_pca_components,
    create_advanced_loss_function,
    pca_transform_targets,
    pca_inverse_transform
)
from src.training_utils import (
    setup_reproducibility,
    create_training_callbacks,
    save_training_artifacts
)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"Project root: {project_root}")

## 2. ⚙️ Configuration and Hyperparameters

### 🎛️ Hyperparameter Philosophy

Proper hyperparameter selection is critical for model performance. Our configuration balances:

1. **Model Capacity**: Sufficient parameters to capture complex IV surface relationships
2. **Regularization**: Prevent overfitting through dropout and advanced loss functions
3. **Training Efficiency**: Optimal batch sizes and learning rates for convergence
4. **Computational Resources**: Memory and time constraints

### 📊 Data Configuration

- **`data_size`**: Dataset size (5k, 100k, 150k samples)
- **`data_format`**: Storage format (`modular` for separate .npy files, `npz` for compressed)

### 🏗️ Architecture Configuration

- **`pca_components`**: Number of principal components (typically 12-30)
  - Reduces output dimensionality from 60 (10 strikes × 6 tenors) to K components
  - Preserves >99.9% of variance while improving training stability
- **`n_blocks`**: Number of residual blocks (depth of network)
- **`width`**: Hidden layer width (model capacity)
- **`dropout_rate`**: Regularization strength (0.0-0.3 typical)

### 🎯 Advanced Loss Function Parameters

Our loss function combines multiple objectives:

$$L_{\text{total}} = L_{\text{Huber}} + \alpha L_{\text{Sobolev}}^{(K)} + \beta L_{\text{Sobolev}}^{(T)} + W_{\text{OTM}} \cdot L_{\text{weighted}}$$

- **`huber_delta`**: Huber loss threshold (robust to outliers)
- **`sobolev_alpha`**: Strike smoothness regularization weight
- **`sobolev_beta`**: Tenor smoothness regularization weight  
- **`otm_put_weight`**: Increased weight for challenging OTM Put region

### 🚂 Training Configuration

- **`epochs`**: Maximum training iterations
- **`batch_size`**: Mini-batch size (affects gradient noise and memory)
- **`learning_rate`**: Initial optimizer learning rate
- **`patience`**: Early stopping patience (prevent overfitting)

In [ ]:
# Training Configuration
config = {
    # Data
    'data_size': '100k',
    'data_format': 'modular',  # or 'npz'
    
    # Model Architecture
    'pca_components': 30,
    'n_blocks': 8,
    'width': 128,
    'dropout_rate': 0.1,
    
    # Loss Function
    'huber_delta': 0.1,
    'sobolev_alpha': 0.01,
    'sobolev_beta': 0.01,
    'otm_put_weight': 2.0,
    
    # Training
    'epochs': 200,
    'batch_size': 256,
    'learning_rate': 0.001,
    'patience': 20,
    
    # Reproducibility
    'random_seed': 42,
    
    # Paths
    'data_path': project_root / 'data' / 'raw' / f'data_{"100k"}',
    'experiments_path': project_root / 'experiments',
    'reports_path': project_root / 'reports',
}

print("Training Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 3. 📊 Data Loading and Preprocessing

### 🔢 Dataset Structure

Our dataset consists of parameter-IV surface pairs generated using FFT-based Heston pricing:

- **Input Features (X)**: 15-dimensional parameter vectors
  - Heston parameters: $(v_0, \kappa, \theta, \sigma, \rho)$
  - Market conditions: $(r)$ (risk-free rate)
  - Contract specifications: $(K_1, ..., K_{10}, T_1, ..., T_6)$ (strikes and tenors)

- **Target Values (y)**: 60-dimensional IV vectors
  - Implied volatility surface: $\text{IV}(K_i, T_j)$ for $i=1...10, j=1...6$
  - Represents the "ground truth" from expensive FFT calculations

### 🎯 Data Split Strategy

We use a standard 60/20/20 train/validation/test split:

- **Training Set**: Model parameter optimization
- **Validation Set**: Hyperparameter tuning and early stopping
- **Test Set**: Final unbiased performance evaluation

### 🔄 Data Normalization

Proper scaling is essential for neural network training:

- **Input Scaling**: StandardScaler (zero mean, unit variance)
  $$X_{\text{scaled}} = \frac{X - \mu_X}{\sigma_X}$$

- **Output Scaling**: MinMaxScaler (bounded range)
  $$y_{\text{scaled}} = \frac{y - y_{\text{min}}}{y_{\text{max}} - y_{\text{min}}}$$

### 📈 Data Quality Insights

We'll examine:
- **Distribution characteristics**: Mean, variance, skewness
- **Range analysis**: Min/max values for sanity checking
- **Missing values**: Data completeness verification